# Target experiments

- [ ] QA (baseline to validate visual component)
- [ ] Caption + QA
- [ ] o4 + Image + QA
- [ ] o4 + Image + Caption + QA

In [ ]:
import torch
from sentence_transformers import SentenceTransformer, CrossEncoder

class VectorStore():
    def __init__(self, embedding_model: SentenceTransformer, rerank_model: CrossEncoder):
        self.embedding_model = embedding_model
        self.rerank_model = rerank_model
        self.vector_store = []
        self.embedding_matrix = None

    def index_documents(self, documents):
        # Encode and normalize all document embeddings
        embeddings = self.embedding_model.encode(
            [doc['payload'] for doc in documents],
            convert_to_tensor=True,
            normalize_embeddings=True
        )

        # Store each document along with its embedding
        self.vector_store = []
        for idx, embedding in enumerate(embeddings):
            self.vector_store.append({
                'id': documents[idx]['id'],
                'payload': documents[idx]['payload'],
                'vector': embedding  # keep it as a torch tensor
            })

        # Stack all embeddings into a single tensor for search
        self.embedding_matrix = torch.stack([entry['vector'] for entry in self.vector_store])

    def search(self, query, top_k=6):
        # Encode and normalize the query
        query_embedding = self.embedding_model.encode(
            [query],
            convert_to_tensor=True,
            normalize_embeddings=True,
            show_progress_bar=False
        )  # shape: (1, dim)

        # Compute cosine similarity (dot product of normalized vectors)
        scores = torch.matmul(query_embedding, self.embedding_matrix.T)  # shape: (1, num_docs)

        # Get top_k scores and corresponding indices
        top_k_scores, top_k_indices = torch.topk(scores, k=top_k, dim=1)

        # Retrieve top_k documents
        top_k_documents = [self.vector_store[idx] for idx in top_k_indices[0].tolist()]

        return top_k_documents, top_k_scores[0].tolist()

    def rerank(self, query, top_k_documents, top_k=6):
        scores = self.rerank_model.rank(
            query=query,
            documents=[doc['payload'] for doc in top_k_documents]
        )
        return [top_k_documents[idx['corpus_id']] for idx in scores][:top_k]


In [ ]:
import pandas as pd

df = pd.read_csv("../data/dataset_V3/corpus.csv", index_col="id")
documents = []
for idx, row in df.iterrows():
    documents.append({"id": idx, "payload": f"{row['title_metadata']} | {row['content']}"})

In [ ]:
import pandas as pd

df = pd.read_csv("../data/dataset_V3/corpus_indicators.csv", index_col="id")
documents_indicators = []
for idx, row in df.iterrows():
    documents_indicators.append({"id": idx, "payload": f"{row['title']} | {row['category']}"})

In [ ]:
vector_store_finetuned = VectorStore(SentenceTransformer('../models/data_trained_V2') , None)
vector_store_finetuned.index_documents(documents)

vector_store_finetuned_indicators = VectorStore(SentenceTransformer('../models/data_trained_V2') , None)
vector_store_finetuned_indicators.index_documents(documents_indicators)

In [ ]:
import pandas as pd

df = pd.read_csv("../data/dataset_V3/corpus_indicators.csv", index_col="id")
documents_indicators_2 = []
for idx, row in df.iterrows():
    documents_indicators_2.append({"id": idx, "payload": f"{row['title']} | {row['category']} | {row['content']}"})

In [ ]:
vector_store_finetuned_indicators_2 = VectorStore(SentenceTransformer('../models/data_trained_V2') , None)
vector_store_finetuned_indicators_2.index_documents(documents_indicators_2)

In [ ]:
from langchain_openai import ChatOpenAI

reasoning = {
    "effort": "medium",  # 'low', 'medium', or 'high'
    "summary": "detailed",  # 'detailed', 'auto', or None
}

llm_o4 = ChatOpenAI(model_name="o4-mini", api_key="", reasoning=reasoning, output_version="responses/v1")

In [ ]:
split_3 = pd.read_csv("../data/dataset_V3/split_3.csv") # has laws + indicators
split_4 = pd.read_csv("../data/dataset_V3/split_4.csv") # has indicators (no laws)

# Init Vector Store & corpus embeddings

In [ ]:


import ast
from tqdm import tqdm
import re
import base64
from langchain_core.messages import HumanMessage
import asyncio
from langchain_openai import ChatOpenAI

MAX_CONCURRENT_REQUESTS = 60
semaphore = asyncio.Semaphore(MAX_CONCURRENT_REQUESTS)

async def mass_runner(data: pd.DataFrame, strategy):
    tasks = []
    for real_idx, (idx, item) in tqdm(enumerate(data.iterrows())):
        # Create tasks for parallel processing
        tasks.append(strategy(item))
    
    results = await asyncio.gather(*tasks)
    final_inputs = []
    final_outputs = []
    final_captions = []
    for result, (idx, item) in zip(results, data.iterrows()):
        input_prompt, output, caption = result
        final_inputs.append(input_prompt)
        final_outputs.append(output)
        final_captions.append(caption)

    data['input_prompt'] = final_inputs
    data['output'] = final_outputs
    data['rephrased_query'] = final_captions

    final_df = data[["id","input_prompt", "output", "rephrased_query"]]
    return final_df

In [ ]:
import pandas as pd

df1 = pd.read_csv("../results/captions-split-3.csv", index_col="id")
df2 = pd.read_csv("../results/captions-split-4.csv", index_col="id")
captions = {}
for idx, row in df1.iterrows():
    captions[idx] = row['captions']
for idx, row in df2.iterrows():
    captions[idx] = row['captions']

In [ ]:
def load_img(href):
    local_url = href.split('/')[-1]
    local_url = local_url.split('.')[0]
    local_url = f"../data/dataset_V3/images/{local_url}.jpg"
    with open(local_url, "rb") as f:
        image_data = base64.b64encode(f.read()).decode("utf-8")
        return image_data
    
def load_caption(id):
    return captions[id]

In [ ]:
# Q + A baseline
async def strategy_1(item):
    answers = " | ".join([entry['answer_text'] 
                                    for entry in ast.literal_eval(item['answers'])])
    return "", "", f'{item['question']} | {answers}'

# Caption + QA
async def strategy_2(item):
    answers = " | ".join([entry['answer_text'] 
                                    for entry in ast.literal_eval(item['answers'])])
    return "", "", f'{load_caption(item['id'])} | {item['question']} | {answers}'

# Image + QA
async def strategy_3(item):
    question = item["question"]
    answers = " | ".join([entry['answer_text'] 
                                    for entry in ast.literal_eval(item['answers'])])
    image = load_img(item['image'])


    full_prompt = """Esti un politist rutier. Vorbesti doar in limba romana.
Primesti o grila de la un test auto care are atasata si o imagine. Trebuie sa selectezi din imagine informatiile necesare,
astfel incat sa imbuntatesti intrebarea originala si a facilita cautarea unor articole de lege relevante.

Include informatiile relevante despre situatie, indicatoare, si altele elemente specifice condusului si legii.

Reguli de gandire:
1. Citeste cu maxima atentie intrebarea si variantele de raspuns.
2. Analizeaza imaginea si extrage informatiile cele mai importante
3. Fii atent la mici detalii care pot schimba sensul intrebarii sau al raspunsurilor
4. Explica ce anume trebuie introdus in intrebare pentru a cauta informatiile corecte


La final, ultima parte din raspuns trebuie sa fie intrebarea reformulata.

"Raspuns final: [intrebare]"  

Aceasta este intrebarea:
{question}

Acestea sunt variantele de raspuns:
{answers}
===================
""".format(question = question, answers = answers)


    message = HumanMessage(
        content=[
            {"type": "text", "text": full_prompt},
            {
                "type": "image_url",
                "image_url": {"url": f"data:image/jpeg;base64,{image}", "detail": "high"},
            }
        ],
    )

    async with semaphore:
        res = await llm_o4.ainvoke([message])
        res = res.content

    reasoning = [x['text'] for x in res[0]['summary']]
    output = res[1]['text']

    output_full = "\n".join([f"[REASONING]{x}" for x in reasoning]) + f"\n[OUTPUT]{output}"
    final_res = output_full.split('final:')[-1].strip()
    final_res = final_res.replace('[','').replace(']','')
    return full_prompt, output_full, final_res

# Image + Caption + QA
async def strategy_4(item):
    question = item["question"]
    answers = " | ".join([entry['answer_text'] 
                                    for entry in ast.literal_eval(item['answers'])])
    image = load_img(item['image'])


    full_prompt = """Esti un politist rutier. Vorbesti doar in limba romana.
Primesti o grila de la un test auto care are atasata si o imagine. Trebuie sa selectezi din imagine informatiile necesare,
astfel incat sa imbuntatesti intrebarea originala si a facilita cautarea unor articole de lege relevante.

Include informatiile relevante despre situatie, indicatoare, si altele elemente specifice condusului si legii.

Reguli de gandire:
1. Citeste cu maxima atentie intrebarea si variantele de raspuns.
2. Analizeaza imaginea si extrage informatiile cele mai importante
3. Fii atent la mici detalii care pot schimba sensul intrebarii sau al raspunsurilor
4. Explica ce anume trebuie introdus in intrebare pentru a cauta informatiile corecte


La final, ultima parte din raspuns trebuie sa fie intrebarea reformulata.

"Raspuns final: [intrebare]"  

Aceasta este intrebarea:
{question}

Acestea sunt variantele de raspuns:
{answers}

Aceasta este descrierea imaginii:
{caption}
===================
""".format(question = question, answers = answers, caption=load_caption(item['id']))


    message = HumanMessage(
        content=[
            {"type": "text", "text": full_prompt},
            {
                "type": "image_url",
                "image_url": {"url": f"data:image/jpeg;base64,{image}", "detail": "high"},
            }
        ],
    )


    async with semaphore:
        res = await llm_o4.ainvoke([message])
        res = res.content

    reasoning = [x['text'] for x in res[0]['summary']]
    output = res[1]['text']

    output_full = "\n".join([f"[REASONING]{x}" for x in reasoning]) + f"\n[OUTPUT]{output}"
    final_res = output_full.split('final:')[-1].strip()
    final_res = final_res.replace('[','').replace(']','')
    return full_prompt, output_full, final_res


In [ ]:
# df1 = pd.read_csv('../data/dataset_V3/split_3.csv')
# df2 = pd.read_csv('../data/dataset_V3/split_4.csv')

# results = await mass_runner(df1, strategy_1)
# results.to_csv('../results/vir/captions_strat_1_split_3.csv', index=False)

# results = await mass_runner(df2, strategy_1)
# results.to_csv('../results/vir/captions_strat_1_split_4.csv', index=False)

In [ ]:
# df1 = pd.read_csv('../data/dataset_V3/split_3.csv')
# df2 = pd.read_csv('../data/dataset_V3/split_4.csv')

# results = await mass_runner(df1, strategy_2)
# results.to_csv('../results/vir/captions_strat_2_split_3.csv', index=False)

# results = await mass_runner(df2, strategy_2)
# results.to_csv('../results/vir/captions_strat_2_split_4.csv', index=False)

In [ ]:
# df1 = pd.read_csv('../data/dataset_V3/split_3.csv')
# df2 = pd.read_csv('../data/dataset_V3/split_4.csv')

# results = await mass_runner(df1, strategy_3)
# results.to_csv('../results/vir/captions_strat_3_split_3.csv', index=False)

# results = await mass_runner(df2, strategy_3)
# results.to_csv('../results/vir/captions_strat_3_split_4.csv', index=False)

In [ ]:
# df1 = pd.read_csv('../data/dataset_V3/split_3.csv')
# df2 = pd.read_csv('../data/dataset_V3/split_4.csv')

# results = await mass_runner(df1, strategy_4)
# results.to_csv('../results/vir/captions_strat_4_split_3.csv', index=False)

# results = await mass_runner(df2, strategy_4)
# results.to_csv('../results/vir/captions_strat_4_split_4.csv', index=False)

# VIR

In [ ]:
import pandas as pd

captions_st1_sp_3 = pd.read_csv('../results/vir/captions_strat_1_split_3.csv')
captions_st1_sp_4 = pd.read_csv('../results/vir/captions_strat_1_split_4.csv')
merged_1 = pd.concat([captions_st1_sp_3, captions_st1_sp_4], ignore_index=True)

In [ ]:
import pandas as pd

captions_st2_sp_3 = pd.read_csv('../results/vir/captions_strat_2_split_3.csv')
captions_st2_sp_4 = pd.read_csv('../results/vir/captions_strat_2_split_4.csv')
merged_2 = pd.concat([captions_st2_sp_3, captions_st2_sp_4], ignore_index=True)

In [ ]:
import pandas as pd

captions_st3_sp_3 = pd.read_csv('../results/vir/captions_strat_3_split_3.csv')
captions_st3_sp_4 = pd.read_csv('../results/vir/captions_strat_3_split_4.csv')
merged_3 = pd.concat([captions_st3_sp_3, captions_st3_sp_4], ignore_index=True)

In [ ]:
import pandas as pd

captions_st4_sp_3 = pd.read_csv('../results/vir/captions_strat_4_split_3.csv')
captions_st4_sp_4 = pd.read_csv('../results/vir/captions_strat_4_split_4.csv')
merged_4 = pd.concat([captions_st4_sp_3, captions_st4_sp_4], ignore_index=True)

In [ ]:
def strategy_1_law(question, possible_answer, id):
    rephrased_question = merged_1.loc[merged_1['id'] == id, 'rephrased_query'].values[0]
    top_k_documents, top_k_scores = vector_store_finetuned.search(f"{rephrased_question}", top_k=10)
    retrieved_set = [doc['id'] for doc in top_k_documents]
    return retrieved_set

def strategy_2_law(question, possible_answer, id):
    rephrased_question = merged_2.loc[merged_2['id'] == id, 'rephrased_query'].values[0]
    top_k_documents, top_k_scores = vector_store_finetuned.search(f"{rephrased_question}", top_k=10)
    retrieved_set = [doc['id'] for doc in top_k_documents]
    return retrieved_set

def strategy_3_law(question, possible_answer, id):
    rephrased_question = merged_3.loc[merged_3['id'] == id, 'rephrased_query'].values[0]
    top_k_documents, top_k_scores = vector_store_finetuned.search(f"{rephrased_question}", top_k=10)
    retrieved_set = [doc['id'] for doc in top_k_documents]
    return retrieved_set

def strategy_4_law(question, possible_answer, id):
    rephrased_question = merged_4.loc[merged_4['id'] == id, 'rephrased_query'].values[0]
    top_k_documents, top_k_scores = vector_store_finetuned.search(f"{rephrased_question}", top_k=10)
    retrieved_set = [doc['id'] for doc in top_k_documents]
    return retrieved_set

def strategy_5_law(question, possible_answer, id):
    rephrased_question = merged_3.loc[merged_3['id'] == id, 'rephrased_query'].values[0]
    top_k_documents, top_k_scores = vector_store_finetuned.search(f"{rephrased_question} | {question} | {possible_answer}", top_k=10)
    retrieved_set = [doc['id'] for doc in top_k_documents]
    return retrieved_set

def strategy_6_law(question, possible_answer, id):
    rephrased_question = merged_4.loc[merged_4['id'] == id, 'rephrased_query'].values[0]
    top_k_documents, top_k_scores = vector_store_finetuned.search(f"{rephrased_question} | {question} | {possible_answer}", top_k=10)
    retrieved_set = [doc['id'] for doc in top_k_documents]
    return retrieved_set

def strategy_1_ind(question, possible_answer, id):
    rephrased_question = merged_1.loc[merged_1['id'] == id, 'rephrased_query'].values[0]
    top_k_documents, top_k_scores = vector_store_finetuned_indicators.search(f"{rephrased_question}", top_k=10)
    retrieved_set = [doc['id'] for doc in top_k_documents]
    return retrieved_set

def strategy_2_ind(question, possible_answer, id):
    rephrased_question = merged_2.loc[merged_2['id'] == id, 'rephrased_query'].values[0]
    top_k_documents, top_k_scores = vector_store_finetuned_indicators.search(f"{rephrased_question}", top_k=10)
    retrieved_set = [doc['id'] for doc in top_k_documents]
    return retrieved_set

def strategy_3_ind(question, possible_answer, id):
    rephrased_question = merged_3.loc[merged_3['id'] == id, 'rephrased_query'].values[0]
    top_k_documents, top_k_scores = vector_store_finetuned_indicators.search(f"{rephrased_question}", top_k=10)
    retrieved_set = [doc['id'] for doc in top_k_documents]
    return retrieved_set

def strategy_4_ind(question, possible_answer, id):
    rephrased_question = merged_4.loc[merged_4['id'] == id, 'rephrased_query'].values[0]
    top_k_documents, top_k_scores = vector_store_finetuned_indicators.search(f"{rephrased_question}", top_k=10)
    retrieved_set = [doc['id'] for doc in top_k_documents]
    return retrieved_set

def strategy_5_ind(question, possible_answer, id):
    rephrased_question = merged_3.loc[merged_3['id'] == id, 'rephrased_query'].values[0]
    top_k_documents, top_k_scores = vector_store_finetuned_indicators.search(f"{rephrased_question} | {question} | {possible_answer}", top_k=10)
    retrieved_set = [doc['id'] for doc in top_k_documents]
    return retrieved_set

def strategy_6_ind(question, possible_answer, id):
    rephrased_question = merged_4.loc[merged_4['id'] == id, 'rephrased_query'].values[0]
    top_k_documents, top_k_scores = vector_store_finetuned_indicators.search(f"{rephrased_question} | {question} | {possible_answer}", top_k=10)
    retrieved_set = [doc['id'] for doc in top_k_documents]
    return retrieved_set

In [ ]:
def strategy_7_ind(question, possible_answer, id):
    rephrased_question = merged_1.loc[merged_1['id'] == id, 'rephrased_query'].values[0]
    top_k_documents, top_k_scores = vector_store_finetuned_indicators_2.search(f"{rephrased_question}", top_k=10)
    retrieved_set = [doc['id'] for doc in top_k_documents]
    return retrieved_set

def strategy_8_ind(question, possible_answer, id):
    rephrased_question = merged_2.loc[merged_2['id'] == id, 'rephrased_query'].values[0]
    top_k_documents, top_k_scores = vector_store_finetuned_indicators_2.search(f"{rephrased_question}", top_k=10)
    retrieved_set = [doc['id'] for doc in top_k_documents]
    return retrieved_set

def strategy_9_ind(question, possible_answer, id):
    rephrased_question = merged_3.loc[merged_3['id'] == id, 'rephrased_query'].values[0]
    top_k_documents, top_k_scores = vector_store_finetuned_indicators_2.search(f"{rephrased_question}", top_k=10)
    retrieved_set = [doc['id'] for doc in top_k_documents]
    return retrieved_set

def strategy_10_ind(question, possible_answer, id):
    rephrased_question = merged_4.loc[merged_4['id'] == id, 'rephrased_query'].values[0]
    top_k_documents, top_k_scores = vector_store_finetuned_indicators_2.search(f"{rephrased_question}", top_k=10)
    retrieved_set = [doc['id'] for doc in top_k_documents]
    return retrieved_set

def strategy_11_ind(question, possible_answer, id):
    rephrased_question = merged_3.loc[merged_3['id'] == id, 'rephrased_query'].values[0]
    top_k_documents, top_k_scores = vector_store_finetuned_indicators_2.search(f"{rephrased_question} | {question} | {possible_answer}", top_k=10)
    retrieved_set = [doc['id'] for doc in top_k_documents]
    return retrieved_set

def strategy_12_ind(question, possible_answer, id):
    rephrased_question = merged_4.loc[merged_4['id'] == id, 'rephrased_query'].values[0]
    top_k_documents, top_k_scores = vector_store_finetuned_indicators_2.search(f"{rephrased_question} | {question} | {possible_answer}", top_k=10)
    retrieved_set = [doc['id'] for doc in top_k_documents]
    return retrieved_set

In [ ]:
import pandas as pd
import ast
from tqdm import tqdm
import pytrec_eval
from collections import defaultdict
import matplotlib.pyplot as plt
from collections.abc import Callable, Awaitable
import numpy as np

def eval_vir_framework_law(dataset: pd.DataFrame, strategy: Callable, name: str, top_k_search=10):
    print('='*30)
    print(f"{name}")
    print('='*30)

    fnqrel = {}
    fqrel = {}
    frun = defaultdict(dict)

    docs_retrieved = []

    for idx, item in tqdm(dataset.iterrows()):
        articles = ast.literal_eval(item['legislation'])  # ground truth: list of relevant articles
        question = item["question"]  # the question for search
        possible_answer = " | ".join([entry['answer_text'] 
                                    for entry in ast.literal_eval(item['answers'])])
        relevant_set = list(set(articles))

        retrieved_set_1 = strategy(question, possible_answer, item['id'])
        docs_retrieved.append(retrieved_set_1)
        qrel = {}
        for doc in relevant_set:
            qrel[doc] = 1
        
        nqrel = {}
        for doc in relevant_set:
            nqrel[doc] = 2

        fqrel[f'{idx}-query'] = qrel
        fnqrel[f'{idx}-query'] = nqrel

        for i in range(1, top_k_search+1):
            run = {}
            for doc, score in zip(retrieved_set_1[:i], [1 - 0.05 * x for x in range(10)]):
                run[doc] = score
            frun[str(i)][f'{idx}-query'] = run
        
        
    recalls = []
    precisions = []
    ndcgs = []
    for i in range(1, top_k_search+1):
        evaluator = pytrec_eval.RelevanceEvaluator(fqrel, {f'recall.{i}'})
        results = evaluator.evaluate(frun[str(i)])
        avg_recall = np.mean([res[f'recall_{i}'] for res in list(results.values())])
        # print(f"Avg Recall@{i}: {avg_recall}")
        recalls.append(avg_recall.item())

        evaluator = pytrec_eval.RelevanceEvaluator(fqrel, {f'P.{i}'})
        results = evaluator.evaluate(frun[str(i)])
        avg_precision = np.mean([res[f'P_{i}'] for res in list(results.values())])
        # print(f"Avg Precision@{i}: {avg_precision}")
        precisions.append(avg_precision.item())

        evaluator = pytrec_eval.RelevanceEvaluator(fnqrel, {f'ndcg_cut.{i}', })
        results = evaluator.evaluate(frun[str(i)])
        avg_ndcg = np.mean([res[f'ndcg_cut_{i}'] for res in list(results.values())])
        # print(f"Avg NDCG@{i}: {avg_ndcg}")
        ndcgs.append(avg_ndcg.item())
    

    dataset['retrieved_documents'] = docs_retrieved
    print("Recall@10", recalls[-1])
    print("Precision@10", precisions[-1])
    print("NDCG@10", ndcgs[-1])
    return recalls, precisions, ndcgs, dataset[['id', 'retrieved_documents']]

    


In [ ]:
import pandas as pd
import ast
from tqdm import tqdm
import pytrec_eval
from collections import defaultdict
import matplotlib.pyplot as plt
from collections.abc import Callable, Awaitable
import numpy as np

def eval_vir_framework_ind(dataset: pd.DataFrame, strategy: Callable, name: str, top_k_search=10):
    print('='*30)
    print(f"{name}")
    print('='*30)

    fnqrel = {}
    fqrel = {}
    frun = defaultdict(dict)

    docs_retrieved = []

    for idx, item in tqdm(dataset.iterrows()):
        articles = ast.literal_eval(item['indicators'])  # ground truth: list of relevant articles
        question = item["question"]  # the question for search
        possible_answer = " | ".join([entry['answer_text'] 
                                    for entry in ast.literal_eval(item['answers'])])
        relevant_set = list(set(articles))

        retrieved_set_1 = strategy(question, possible_answer, item['id'])
        docs_retrieved.append(retrieved_set_1)
        qrel = {}
        for doc in relevant_set:
            qrel[doc] = 1
        
        nqrel = {}
        for doc in relevant_set:
            nqrel[doc] = 2

        fqrel[f'{idx}-query'] = qrel
        fnqrel[f'{idx}-query'] = nqrel

        for i in range(1, top_k_search+1):
            run = {}
            for doc, score in zip(retrieved_set_1[:i], [1 - 0.05 * x for x in range(10)]):
                run[doc] = score
            frun[str(i)][f'{idx}-query'] = run
        
        
    recalls = []
    precisions = []
    ndcgs = []
    for i in range(1, top_k_search+1):
        evaluator = pytrec_eval.RelevanceEvaluator(fqrel, {f'recall.{i}'})
        results = evaluator.evaluate(frun[str(i)])
        avg_recall = np.mean([res[f'recall_{i}'] for res in list(results.values())])
        # print(f"Avg Recall@{i}: {avg_recall}")
        recalls.append(avg_recall.item())

        evaluator = pytrec_eval.RelevanceEvaluator(fqrel, {f'P.{i}'})
        results = evaluator.evaluate(frun[str(i)])
        avg_precision = np.mean([res[f'P_{i}'] for res in list(results.values())])
        # print(f"Avg Precision@{i}: {avg_precision}")
        precisions.append(avg_precision.item())

        evaluator = pytrec_eval.RelevanceEvaluator(fnqrel, {f'ndcg_cut.{i}', })
        results = evaluator.evaluate(frun[str(i)])
        avg_ndcg = np.mean([res[f'ndcg_cut_{i}'] for res in list(results.values())])
        # print(f"Avg NDCG@{i}: {avg_ndcg}")
        ndcgs.append(avg_ndcg.item())
    

    dataset['retrieved_documents'] = docs_retrieved
    print("Recall@10", recalls[-1])
    print("Precision@10", precisions[-1])
    print("NDCG@10", ndcgs[-1])
    return recalls, precisions, ndcgs, dataset[['id', 'retrieved_documents']]

- split 3 laws
- split 3 inds
- split 4 inds

In [ ]:
df_split_3 = pd.read_csv('../data/dataset_V3/split_3.csv')
recalls, precisions, ndcgs, retrieved = eval_vir_framework_law(df_split_3, strategy_1_law, '3-law')
print(recalls)
print(precisions)
print(ndcgs)
retrieved.to_csv('../results/vir/vir_strat_1_split_3_laws.csv', index=False)

df_split_3 = pd.read_csv('../data/dataset_V3/split_3.csv')
recalls, precisions, ndcgs, retrieved = eval_vir_framework_ind(df_split_3, strategy_1_ind, '3-ind')
print(recalls)
print(precisions)
print(ndcgs)
retrieved.to_csv('../results/vir/vir_strat_1_split_3_ind.csv', index=False)

df_split_4 = pd.read_csv('../data/dataset_V3/split_4.csv')
recalls, precisions, ndcgs, retrieved = eval_vir_framework_ind(df_split_4, strategy_1_ind, '4-ind')
print(recalls)
print(precisions)
print(ndcgs)
retrieved.to_csv('../results/vir/vir_strat_1_split_4_ind.csv', index=False)
##########################################



In [ ]:
df_split_3 = pd.read_csv('../data/dataset_V3/split_3.csv')
recalls, precisions, ndcgs, retrieved = eval_vir_framework_law(df_split_3, strategy_2_law, '3-law')
print(recalls)
print(precisions)
print(ndcgs)
retrieved.to_csv('../results/vir/vir_strat_2_split_3_laws.csv', index=False)

df_split_3 = pd.read_csv('../data/dataset_V3/split_3.csv')
recalls, precisions, ndcgs, retrieved = eval_vir_framework_ind(df_split_3, strategy_2_ind, '3-ind')
print(recalls)
print(precisions)
print(ndcgs)
retrieved.to_csv('../results/vir/vir_strat_2_split_3_ind.csv', index=False)

df_split_4 = pd.read_csv('../data/dataset_V3/split_4.csv')
recalls, precisions, ndcgs, retrieved = eval_vir_framework_ind(df_split_4, strategy_2_ind, '4-ind')
print(recalls)
print(precisions)
print(ndcgs)
retrieved.to_csv('../results/vir/vir_strat_2_split_4_ind.csv', index=False)
##########################################



In [ ]:
df_split_3 = pd.read_csv('../data/dataset_V3/split_3.csv')
recalls, precisions, ndcgs, retrieved = eval_vir_framework_law(df_split_3, strategy_3_law, '3-law')
print(recalls)
print(precisions)
print(ndcgs)
retrieved.to_csv('../results/vir/vir_strat_3_split_3_laws.csv', index=False)

df_split_3 = pd.read_csv('../data/dataset_V3/split_3.csv')
recalls, precisions, ndcgs, retrieved = eval_vir_framework_ind(df_split_3, strategy_3_ind, '3-ind')
print(recalls)
print(precisions)
print(ndcgs)
retrieved.to_csv('../results/vir/vir_strat_3_split_3_ind.csv', index=False)

df_split_4 = pd.read_csv('../data/dataset_V3/split_4.csv')
recalls, precisions, ndcgs, retrieved = eval_vir_framework_ind(df_split_4, strategy_3_ind, '4-ind')
print(recalls)
print(precisions)
print(ndcgs)
retrieved.to_csv('../results/vir/vir_strat_3_split_4_ind.csv', index=False)
##########################################



In [ ]:
df_split_3 = pd.read_csv('../data/dataset_V3/split_3.csv')
recalls, precisions, ndcgs, retrieved = eval_vir_framework_law(df_split_3, strategy_4_law, '3-law')
print(recalls)
print(precisions)
print(ndcgs)
retrieved.to_csv('../results/vir/vir_strat_4_split_3_laws.csv', index=False)

df_split_3 = pd.read_csv('../data/dataset_V3/split_3.csv')
recalls, precisions, ndcgs, retrieved = eval_vir_framework_ind(df_split_3, strategy_4_ind, '3-ind')
print(recalls)
print(precisions)
print(ndcgs)
retrieved.to_csv('../results/vir/vir_strat_4_split_3_ind.csv', index=False)

df_split_4 = pd.read_csv('../data/dataset_V3/split_4.csv')
recalls, precisions, ndcgs, retrieved = eval_vir_framework_ind(df_split_4, strategy_4_ind, '4-ind')
print(recalls)
print(precisions)
print(ndcgs)
retrieved.to_csv('../results/vir/vir_strat_4_split_4_ind.csv', index=False)
##########################################



In [ ]:
df_split_4 = pd.read_csv('../data/dataset_V3/split_4.csv')
recalls, precisions, ndcgs, retrieved = eval_vir_framework_law(df_split_4, strategy_5_law, '4-law')
print(recalls)
print(precisions)
print(ndcgs)
retrieved.to_csv('../results/vir/vir_strat_5_split_4_laws.csv', index=False)

In [ ]:
df_split_3 = pd.read_csv('../data/dataset_V3/split_3.csv')
recalls, precisions, ndcgs, retrieved = eval_vir_framework_law(df_split_3, strategy_5_law, '3-law')
print(recalls)
print(precisions)
print(ndcgs)
retrieved.to_csv('../results/vir/vir_strat_5_split_3_laws.csv', index=False)

df_split_3 = pd.read_csv('../data/dataset_V3/split_3.csv')
recalls, precisions, ndcgs, retrieved = eval_vir_framework_ind(df_split_3, strategy_5_ind, '3-ind')
print(recalls)
print(precisions)
print(ndcgs)
retrieved.to_csv('../results/vir/vir_strat_5_split_3_ind.csv', index=False)

df_split_4 = pd.read_csv('../data/dataset_V3/split_4.csv')
recalls, precisions, ndcgs, retrieved = eval_vir_framework_ind(df_split_4, strategy_5_ind, '4-ind')
print(recalls)
print(precisions)
print(ndcgs)
retrieved.to_csv('../results/vir/vir_strat_5_split_4_ind.csv', index=False)
##########################################



In [ ]:
df_split_3 = pd.read_csv('../data/dataset_V3/split_3.csv')
recalls, precisions, ndcgs, retrieved = eval_vir_framework_law(df_split_3, strategy_6_law, '3-law')
print(recalls)
print(precisions)
print(ndcgs)
retrieved.to_csv('../results/vir/vir_strat_6_split_3_laws.csv', index=False)

df_split_3 = pd.read_csv('../data/dataset_V3/split_3.csv')
recalls, precisions, ndcgs, retrieved = eval_vir_framework_ind(df_split_3, strategy_6_ind, '3-ind')
print(recalls)
print(precisions)
print(ndcgs)
retrieved.to_csv('../results/vir/vir_strat_6_split_3_ind.csv', index=False)

df_split_4 = pd.read_csv('../data/dataset_V3/split_4.csv')
recalls, precisions, ndcgs, retrieved = eval_vir_framework_ind(df_split_4, strategy_6_ind, '4-ind')
print(recalls)
print(precisions)
print(ndcgs)
retrieved.to_csv('../results/vir/vir_strat_6_split_4_ind.csv', index=False)
##########################################



In [ ]:
df_split_3 = pd.read_csv('../data/dataset_V3/split_3.csv')
recalls, precisions, ndcgs, retrieved = eval_vir_framework_ind(df_split_3, strategy_7_ind, '3-ind')
print(recalls)
print(precisions)
print(ndcgs)
retrieved.to_csv('../results/vir/vir_strat_7_split_3_ind.csv', index=False)

df_split_4 = pd.read_csv('../data/dataset_V3/split_4.csv')
recalls, precisions, ndcgs, retrieved = eval_vir_framework_ind(df_split_4, strategy_7_ind, '4-ind')
print(recalls)
print(precisions)
print(ndcgs)
retrieved.to_csv('../results/vir/vir_strat_7_split_4_ind.csv', index=False)

In [ ]:
df_split_3 = pd.read_csv('../data/dataset_V3/split_3.csv')
recalls, precisions, ndcgs, retrieved = eval_vir_framework_ind(df_split_3, strategy_8_ind, '3-ind')
print(recalls)
print(precisions)
print(ndcgs)
retrieved.to_csv('../results/vir/vir_strat_8_split_3_ind.csv', index=False)

df_split_4 = pd.read_csv('../data/dataset_V3/split_4.csv')
recalls, precisions, ndcgs, retrieved = eval_vir_framework_ind(df_split_4, strategy_8_ind, '4-ind')
print(recalls)
print(precisions)
print(ndcgs)
retrieved.to_csv('../results/vir/vir_strat_8_split_4_ind.csv', index=False)

In [ ]:
df_split_3 = pd.read_csv('../data/dataset_V3/split_3.csv')
recalls, precisions, ndcgs, retrieved = eval_vir_framework_ind(df_split_3, strategy_9_ind, '3-ind')
print(recalls)
print(precisions)
print(ndcgs)
retrieved.to_csv('../results/vir/vir_strat_9_split_3_ind.csv', index=False)

df_split_4 = pd.read_csv('../data/dataset_V3/split_4.csv')
recalls, precisions, ndcgs, retrieved = eval_vir_framework_ind(df_split_4, strategy_9_ind, '4-ind')
print(recalls)
print(precisions)
print(ndcgs)
retrieved.to_csv('../results/vir/vir_strat_9_split_4_ind.csv', index=False)

In [ ]:
df_split_3 = pd.read_csv('../data/dataset_V3/split_3.csv')
recalls, precisions, ndcgs, retrieved = eval_vir_framework_ind(df_split_3, strategy_10_ind, '3-ind')
print(recalls)
print(precisions)
print(ndcgs)
retrieved.to_csv('../results/vir/vir_strat_10_split_3_ind.csv', index=False)

df_split_4 = pd.read_csv('../data/dataset_V3/split_4.csv')
recalls, precisions, ndcgs, retrieved = eval_vir_framework_ind(df_split_4, strategy_10_ind, '4-ind')
print(recalls)
print(precisions)
print(ndcgs)
retrieved.to_csv('../results/vir/vir_strat_10_split_4_ind.csv', index=False)

In [ ]:
df_split_3 = pd.read_csv('../data/dataset_V3/split_3.csv')
recalls, precisions, ndcgs, retrieved = eval_vir_framework_ind(df_split_3, strategy_11_ind, '3-ind')
print(recalls)
print(precisions)
print(ndcgs)
retrieved.to_csv('../results/vir/vir_strat_11_split_3_ind.csv', index=False)

df_split_4 = pd.read_csv('../data/dataset_V3/split_4.csv')
recalls, precisions, ndcgs, retrieved = eval_vir_framework_ind(df_split_4, strategy_11_ind, '4-ind')
print(recalls)
print(precisions)
print(ndcgs)
retrieved.to_csv('../results/vir/vir_strat_11_split_4_ind.csv', index=False)

In [ ]:
df_split_3 = pd.read_csv('../data/dataset_V3/split_3.csv')
recalls, precisions, ndcgs, retrieved = eval_vir_framework_ind(df_split_3, strategy_12_ind, '3-ind')
print(recalls)
print(precisions)
print(ndcgs)
retrieved.to_csv('../results/vir/vir_strat_12_split_3_ind.csv', index=False)

df_split_4 = pd.read_csv('../data/dataset_V3/split_4.csv')
recalls, precisions, ndcgs, retrieved = eval_vir_framework_ind(df_split_4, strategy_12_ind, '4-ind')
print(recalls)
print(precisions)
print(ndcgs)
retrieved.to_csv('../results/vir/vir_strat_12_split_4_ind.csv', index=False)